# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pprint import pprint

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,west island,-12.1568,96.8225,28.99,84,40,6.69,CC,1741229564
1,1,margaret river,-33.9500,115.0667,29.09,40,80,3.23,AU,1741229565
2,2,grytviken,-54.2811,-36.5092,5.34,95,78,5.19,GS,1741229566
3,3,blackmans bay,-43.0167,147.3167,17.63,59,49,3.08,AU,1741229568
4,4,yellowknife,62.4560,-114.3525,-20.71,84,20,1.03,CA,1741229569


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [9]:
%%capture --no-display

# Configure the map plot (5)
#formatting so that the size of each data point correlates to the humidity in the associated city. (6)
map_plot_1 = city_data_df.hvplot.points("Lng", 
                                        "Lat", 
                                        geo=True, 
                                        tiles="OSM",
                                        xlim = (-180,180),
                                       size = "Humidity",
                                       color = "City",
                                       scale = 0.7)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [10]:
# Narrow down cities that fit criteria and drop any results with null values (7)
good_weather_df = city_data_df.loc[(city_data_df["Max Temp"]>= 21) & (city_data_df["Max Temp"]<= 27) & (city_data_df["Wind Speed"]<= 4.5) & (city_data_df["Cloudiness"]== 0)]

# Drop any rows with null values (8)
good_weather_df = good_weather_df.dropna(how='any')

# Display sample data
good_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
107,107,arraial do cabo,-22.9661,-42.0278,25.16,86,0,4.38,BR,1741229702
169,169,tura,25.5198,90.2201,25.60,19,0,0.84,IN,1741229775
245,245,dondo,-19.6094,34.7431,24.73,95,0,0.54,MZ,1741229869
279,279,sylhet,24.5000,91.6667,25.91,32,0,3.13,BD,1741229912
421,421,kerikeri,-35.2268,173.9474,21.48,40,0,3.91,NZ,1741230095


### Step 3: Create a new DataFrame called `hotel_df`.

In [11]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity (9)
hotel_df = good_weather_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API (10)
hotel_df["Hotel Name"] = ""

#reset index, for improved iteration in following steps (11)
hotel_df = hotel_df.reset_index(drop=True)

# Display sample data
hotel_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,107,arraial do cabo,-22.9661,-42.0278,25.16,86,0,4.38,BR,1741229702,
1,169,tura,25.5198,90.2201,25.60,19,0,0.84,IN,1741229775,
2,245,dondo,-19.6094,34.7431,24.73,95,0,0.54,MZ,1741229869,
3,279,sylhet,24.5000,91.6667,25.91,32,0,3.13,BD,1741229912,
4,421,kerikeri,-35.2268,173.9474,21.48,40,0,3.91,NZ,1741230095,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [12]:
# Set parameters to search for a hotel
# sources for this code block (12) (13)
radius = 10000
params = {"categories":"accommodation.hotel",
         "apiKey":geoapify_key}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = hotel_df.iloc[index,2]
    lon = hotel_df.iloc[index,3]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url,params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
arraial do cabo - nearest hotel: No hotel found
tura - nearest hotel: Hotel Polo Orchid
dondo - nearest hotel: No hotel found
sylhet - nearest hotel: Rest Inn Hotel
kerikeri - nearest hotel: Avalon Resort
rajbiraj - nearest hotel: Deep Shajan Hotel & Lodge
arangaon - nearest hotel: No hotel found
visnagar - nearest hotel: No hotel found
toliara - nearest hotel: Ambary


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,107,arraial do cabo,-22.9661,-42.0278,25.16,86,0,4.38,BR,1741229702,No hotel found
1,169,tura,25.5198,90.2201,25.60,19,0,0.84,IN,1741229775,Hotel Polo Orchid
2,245,dondo,-19.6094,34.7431,24.73,95,0,0.54,MZ,1741229869,No hotel found
3,279,sylhet,24.5000,91.6667,25.91,32,0,3.13,BD,1741229912,Rest Inn Hotel
4,421,kerikeri,-35.2268,173.9474,21.48,40,0,3.91,NZ,1741230095,Avalon Resort
5,506,rajbiraj,26.5333,86.7500,21.52,31,0,2.35,NP,1741230201,Deep Shajan Hotel & Lodge
6,566,arangaon,18.6667,75.1833,23.25,10,0,3.90,IN,1741230277,No hotel found
7,568,visnagar,23.7000,72.5500,21.94,9,0,1.84,IN,1741230279,No hotel found
8,573,toliara,-23.3500,43.6667,25.03,86,0,2.01,MG,1741230285,Ambary


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [13]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points("Lng",
                                    "Lat",
                                    geo=True,
                                    tiles="OSM",
                                    xlim=(-180,180),
                                    ylim=(-70,70),
                                    size="Humidity",
                                    color="City",
                                    scale=2,
                                    hover_cols=["Hotel Name", "Country"] #(14)
                                   )

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)